In [1]:
from pprint import pprint
from typing import Literal, List, Dict

from gen_ai_hub.proxy.native.openai import chat, completions
from pipeline.generate_tool_schema import generate_json_schema

In [2]:
################################
# Define Your Tool functions Here

def cosine(x: int) -> float:
    """
    Calculate the cosine of an integer 
    
    :param x: The input integer
    :return: A float representing the cosine of x
    """
    import math
    return math.cos(x)

def sin(x: int) -> float:
    """
    Calculate the sine of an integer

    :param x: The input integer
    :return: A float representing the sine of x
    """
    import math
    return math.sin(x)

def addition(x: float, y: float) -> float:
    """
    Calculate the addition of two float numbers

    :param x: The input float x
    :param y: The input float y
    :return: A float representing the sum of x and y
    """
    return x + y

In [8]:
def run_loop(prompt, tools, function_mapping):
    import json
    continue_loop = True
    
    messages = []
    system_message = {"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."}
    user_message =  {"role": "user", "content": prompt}
    messages.append(system_message)
    messages.append(user_message)

    while continue_loop: 
        response = chat.completions.create( 
            messages = messages,
            model_name = "gpt-4",
            tools=tools)
        print(response) 
        print('\n')
        response_message = response.choices[0].message
        
        if response.choices[0].finish_reason == "stop":
            return response_message.content

        tool_call_block = response_message.tool_calls[0]
        if tool_call_block:
            messages.append({"role": "assistant", "tool_calls": response_message.tool_calls})
            tool_call_id = tool_call_block.id
            tool_call_function_name = tool_call_block.function.name
            tool_call_function_arguments = json.loads(tool_call_block.function.arguments)
            tool_call_function = function_mapping.get(tool_call_function_name)
            tool_call_result = tool_call_function(**tool_call_function_arguments)
            function_call_result_message = {
                 "role" : "tool", 
                 "content": json.dumps({**tool_call_function_arguments, "result": tool_call_result}),
                 "tool_call_id": tool_call_id
            }
            messages.append(function_call_result_message) 



In [9]:
tools = []
tools.append(generate_json_schema(cosine))
tools.append(generate_json_schema(sin))
tools.append(generate_json_schema(addition))

function_mapping = {
    "cosine": cosine, 
    "sin": sin,
    "addition": addition
}

run_loop(prompt = "Calculate cos(3) + sin(4) for me", 
         tools = tools, 
         function_mapping = function_mapping)

ChatCompletion(id='chatcmpl-AaSdLuAO4zvt55w0ClcquX5NFNv9i', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_P52DRp0dIqWsIxA3nuQ3qHZM', function=Function(arguments='{"x":3}', name='cosine'), type='function')]), content_filter_results={})], created=1733253239, model='gpt-4-turbo-2024-04-09', object='chat.completion', service_tier=None, system_fingerprint='fp_5603ee5e2e', usage=CompletionUsage(completion_tokens=14, prompt_tokens=145, total_tokens=159, completion_tokens_details=None, prompt_tokens_details=None), prompt_filter_results=[{'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}, 'prompt_index': 0}])


ChatCompletion(id='chatcm

'The result of \\( \\cos(3) + \\sin(4) \\) is approximately -1.7468.'